# Outil de débogage interactif

**Description**

Ce notebook offre un outil interactif (widget) pour visualiser les évènements des matchs répertoriés par saison.

In [1]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
import json
import pprint

In [2]:
@interact
def interactive_debugger(season=['2016_2017', '2017_2018', '2018_2019', '2019_2020', '2020_2021'], season_type=['regular', 'playoff']):
    """
    season: str
        Season to visualize. 
    season_type: str
        Season type to visualize. Can be `reguliere` or `eliminatoire`.
    gameId: int
        Game number to visualize. 0 corresponds to the first game of the season.
    eventIdx: int
        Event number to visualize within the specified game. 0 corresponds to the first event of the game.
    """
    # TODO :integrate `season_type` 
 
    # Get data
    info = json.load(open(f'../ift6758/data/data_saison_{season}_play_by_play.json'))

    # Widget with Game ID
    @interact
    def interact_game(gameId = (0, len(info)-1, 1)):
        # Widget with Event ID
        print(info[gameId][0]['about']['dateTime'])
        print(f'Game ID: {gameId}; (away) vs (home)')
        @interact
        def interact_event(eventIdx = (0, len(info[gameId])-1, 1)):
            # Retrieve coord for specified `event` in `game_id`
            coord = info[gameId][eventIdx]['coordinates']
            # Plot if coord is not empty
            im = plt.imread('../figures/nhl_rink.png')
            fig, ax = plt.subplots(figsize=(20, 15))
            # Adjust figure size to match rink dimensions
            height_rink, width_rink = 85, 200 # NHL rink dimensions
            ax.imshow(im, extent=[-width_rink/2, width_rink/2, -height_rink/2, height_rink/2])
            if coord:
                ax.scatter(coord['x'], coord['y'], s=300, c='blue')
            plt.xlabel('feet')
            plt.ylabel('feet')
            plt.title(f"{info[gameId][eventIdx]['result']['description']}\n{info[gameId][eventIdx]['about']['periodTime']} P-{info[gameId][eventIdx]['about']['period']}", fontsize=15)
            plt.show()
            pprint.pprint(info[gameId][eventIdx])
            


interactive(children=(Dropdown(description='season', options=('2016_2017', '2017_2018', '2018_2019', '2019_202…